In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import numpy as np

In [9]:
spark = SparkSession.builder.appName("SeriesTemporales").getOrCreate()
# Leer archivo Parquet
df_temp_media_mad = spark.read.parquet("../../data/homogenized/diarios/indicativo=3195/tipo_dato=temperatura_media")
# Mostrar datos
#df_temp_media.show()
df_temp_media_mad.show()

+----------+------------------+----+---+
|     fecha|             value|anio|mes|
+----------+------------------+----+---+
|2020-08-01|              29.5|2020|  8|
|2020-08-02|28.600000381469727|2020|  8|
|2020-08-03|              24.0|2020|  8|
|2020-08-04|26.100000381469727|2020|  8|
|2020-08-05|27.799999237060547|2020|  8|
|2020-08-06|30.799999237060547|2020|  8|
|2020-08-07|31.700000762939453|2020|  8|
|2020-08-08|29.799999237060547|2020|  8|
|2020-08-09|31.200000762939453|2020|  8|
|2020-08-10|26.100000381469727|2020|  8|
|2020-08-11|19.700000762939453|2020|  8|
|2020-08-12|20.899999618530273|2020|  8|
|2020-08-13|23.200000762939453|2020|  8|
|2020-08-14|25.600000381469727|2020|  8|
|2020-08-15|23.399999618530273|2020|  8|
|2020-08-16|21.799999237060547|2020|  8|
|2020-08-17|21.600000381469727|2020|  8|
|2020-08-18|              24.5|2020|  8|
|2020-08-19|25.399999618530273|2020|  8|
|2020-08-20|25.399999618530273|2020|  8|
+----------+------------------+----+---+
only showing top

In [10]:
df_temp_media_mad.count()

2233

In [ ]:
df_pandas = df_temp_media.toPandas()

In [ ]:
df_pandas["dia"] = df_pandas["fecha"].dt.day
#df_pandas.sort_values(by="fecha", inplace=True)
#df_pandas.reset_index(drop=True, inplace=True)

In [ ]:
df_pandas['sin_dia'] = np.sin(2 * np.pi * df_pandas['dia'] / 365)
df_pandas['cos_dia'] = np.cos(2 * np.pi * df_pandas['dia'] / 365)

df_pandas['sin_mes'] = np.sin(2 * np.pi * df_pandas['mes'] / 12)
df_pandas['cos_mes'] = np.cos(2 * np.pi * df_pandas['mes'] / 12)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

In [ ]:
max_p = 3 # You can try up to 5 more is not recommended
max_d = 3 # You can try up to 5 more is not recommended
max_q = 3 # You can try up to 5 more is not recommended
max_P = 2 # You can try up to 3 more is not recommended 
max_D = 2 # You can try up to 3 more is not recommended
max_Q = 2 # You can try up to 3 more is not recommended

seasonality_order = 12 #because is a monthly data and we have seen yearly seasonality

model = auto_arima(df_pandas['value'], 
                   seasonal=True, 
                   m=12, 
                   trace=True, 
                   suppress_warnings=True, 
                   stepwise=True,
                   max_p=max_p, max_d=max_d, max_q=max_q,
                   max_P=max_P, max_D=max_D, max_Q=max_Q,
                   n_jobs=-1)

In [ ]:
exog_variables = ['sin_dia', 'cos_dia', 'sin_mes', 'cos_mes']

X = df_pandas[exog_variables]
y = df_pandas['value']

#X = X.astype(float)
#y = y.astype(float)

modelo = SARIMAX(y, exog=X, order=(p, d, q), seasonal_order=(P, D, Q, 12))
modelo.fit()
print(modelo.summary())

In [ ]:
forecast = modelo.forecast(steps=7, exog=X)
print(forecast)